<a href="https://colab.research.google.com/github/sukhyun1017/Data-mining/blob/main/psy_text_mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
get_ipython().system('python -m spacy download en_core_web_sm ')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 15.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [3]:
import en_core_web_sm

nlp = en_core_web_sm.load()

In [4]:
import numpy as np
import pandas as pd
import re, nltk, spacy, gensim
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint


In [6]:
df = pd.read_csv("psy2.csv", engine = 'python', encoding='cp949')
df = df.dropna(subset=["comment"])


In [7]:
df.head()

,youtube_id,comment,date,like_num
0,So Hyang TV ? Fan Channel,"PSY comeback 4/29, BIGBANG comeback, and now 2...",1개월 전,1만
1,Harley Dickson,"This guy appeared out of nowhere, and everyone...",2년 전,2.6만
2,Zef Davenport,"Lets be honest here, this guy owned the world ...",2년 전(수정됨),11만
3,myz1c,Didn't realize we have been listening to kpop ...,2년 전(수정됨),21만
4,Natalia Lidiane,Saudades dessa ?poca! Aeee cad? os BR??? ?????...,2년 전,2.2만


In [8]:
data = df.comment.values.tolist()
# Emails 제거 
data = [re.sub(r'\S*@\S*\s?', '', sent) for sent in data]
# new line characters 제거
data = [re.sub(r'\s+', ' ', sent) for sent in data]
# distracting single quotes 제거
data = [re.sub(r"\'", "", sent) for sent in data]
pprint(data[:3])

['PSY comeback 4/29, BIGBANG comeback, and now 2NE1 reunion. 2nd gen rise??',
 'This guy appeared out of nowhere, and everyone just accepted it, they didnt '
 'question it, just accepted it and bowed down.',
 'Lets be honest here, this guy owned the world for a while.Well Im glad 115k '
 'people agree on this. Maybe he still owns the world.']


In [9]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # punctuations 제거 
        
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']): #'NOUN', 'Adjective', 'VERB', 'Adverb'
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

data_words = list(sent_to_words(data))
print(data_words[:1])

nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
data_lemmatized = lemmatization(data_words, allowed_postags=["NOUN", "VERB"]) #명사랑 동사 고르기
print(data_lemmatized[:3])

[['psy', 'comeback', 'bigbang', 'comeback', 'and', 'now', 'ne', 'reunion', 'nd', 'gen', 'rise']]
['rise', 'guy appear accept question accept bow', 'let guy own world m people agree own world']


In [10]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,
                             stop_words='english',             
                             lowercase=True,                   
                             token_pattern='[a-zA-Z0-9]{3,}') 
data_vectorized = vectorizer.fit_transform(data_lemmatized)

In [11]:
lda_model = LatentDirichletAllocation(n_components=20,max_iter=10,learning_method='online',random_state=100,batch_size=128,evaluate_every = -1,n_jobs = -1,               )
lda_output = lda_model.fit_transform(data_vectorized)
print(lda_model) 

LatentDirichletAllocation(learning_method='online', n_components=20, n_jobs=-1,
                          random_state=100)


In [12]:
LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
 evaluate_every=-1, learning_decay=0.7,
 learning_method="online", learning_offset=10.0,
 max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
 n_components=10, n_jobs=-1, perp_tol=0.1,
 random_state=100, topic_word_prior=None,
 total_samples=1000000.0, verbose=0)

LatentDirichletAllocation(learning_method='online', n_jobs=-1, random_state=100)

In [13]:
print("Log Likelihood: ", lda_model.score(data_vectorized))

print("Perplexity: ", lda_model.perplexity(data_vectorized))

pprint(lda_model.get_params())

Log Likelihood:  -713343.2022601041
Perplexity:  237.85997324017788
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 20,
 'n_jobs': -1,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


In [16]:
# Search Parameter 찾기.
search_params = {'n_components': [10,15,20], 'learning_decay': [0.5,0.7, 0.9]}

model = GridSearchCV(lda_model, param_grid=search_params)

model.fit(data_vectorized)

GridSearchCV(estimator=LatentDirichletAllocation(learning_method='online',
                                                 n_components=20, n_jobs=-1,
                                                 random_state=100),
             param_grid={'learning_decay': [0.5, 0.7, 0.9],
                         'n_components': [10, 15, 20]})

In [17]:
# Best Model
best_lda_model = model.best_estimator_
# Model Parameters
print("Best Model's Params: ", model.best_params_)
# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)
# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

Best Model's Params:  {'learning_decay': 0.9, 'n_components': 10}
Best Log Likelihood Score:  -145401.38371698838
Model Perplexity:  210.31444597336113


In [18]:
# Create Document — Topic Matrix
lda_output = best_lda_model.transform(data_vectorized)

topicnames = ["Topic" + str(i) for i in range(best_lda_model.n_components)]
docnames = ["Doc" + str(i) for i in range(len(data))]

# pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)
# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic["dominant_topic"] = dominant_topic
# Styling
def color_green(val):
 color = "green" if val > .1 else "black"
 return "color: {col}".format(col=color)
def make_bold(val):
 weight = 700 if val > .1 else 400
 return "font-weight: {weight}".format(weight=weight)
# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics


,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,dominant_topic
Doc0,0.050000,0.050000,0.050000,0.550000,0.050000,0.050000,0.050000,0.050000,0.050000,0.050000,3
Doc1,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.870000,0.010000,0.010000,7
Doc2,0.010000,0.010000,0.010000,0.010000,0.010000,0.140000,0.010000,0.010000,0.160000,0.610000,9
Doc3,0.020000,0.820000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,1
Doc4,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.770000,0.030000,8
Doc5,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.700000,0.030000,0.030000,0.030000,6
Doc6,0.030000,0.030000,0.030000,0.030000,0.770000,0.030000,0.030000,0.030000,0.030000,0.030000,4
Doc7,0.010000,0.770000,0.010000,0.010000,0.010000,0.080000,0.010000,0.010000,0.090000,0.010000,1
Doc8,0.030000,0.030000,0.700000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,0.030000,2
Doc9,0.020000,0.420000,0.020000,0.020000,0.020000,0.420000,0.020000,0.020000,0.020000,0.020000,1


In [19]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(best_lda_model.components_)
# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames
# View
df_topic_keywords.head()

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aaaa,aburre,aburrir,accept,account,achieve,achievement,add,addict,addictive,...,wtf,yard,year,yell,yeni,yesterday,youth,youtube,yrs,zaman
Topic0,0.100038,0.100048,0.101789,0.100049,0.131638,30.403471,0.100044,0.100054,0.100059,0.101985,...,0.100076,0.100046,17.304918,0.100040,0.100042,0.100069,0.100772,0.103766,8.777850,0.100049
Topic1,0.100052,0.100049,0.100042,0.100047,0.100034,0.100049,0.100049,0.107869,46.877650,0.100392,...,0.100055,0.100051,4971.455947,0.100041,0.100051,172.149622,0.100348,0.100114,2.094770,0.100043
Topic2,16.135121,0.100056,0.100053,0.100045,0.100041,0.100051,0.101863,38.401365,0.100057,0.100310,...,0.100051,0.152554,0.100108,0.603125,0.100049,0.100468,0.100678,0.100156,0.100041,0.100055
Topic3,0.100044,0.100045,0.100043,0.100056,0.100042,0.100068,0.100041,0.100037,0.100048,0.100148,...,0.100036,0.100041,495.199569,0.100050,0.100043,0.100075,0.100063,0.100217,0.100062,0.100042
Topic4,0.100044,0.100047,0.100040,0.100052,0.100069,0.100509,0.103632,0.100048,0.100086,0.100049,...,0.100397,15.114754,0.767027,14.542645,0.100051,0.100056,0.100172,65.295378,0.100052,0.100050


In [20]:
#각 토픽의 대표 키워드 추출. 
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords
topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=15)
# DF
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,watch,style,make,memory,think,dance,video,bring,thank,este,imagine,psy,remember,change,banger
Topic 1,year,song,kpop,listen,know,use,dance,believe,remember,release,miss,time,today,day,lot
Topic 2,psy,comment,hit,style,girl,look,thing,lady,want,styleeh,end,baby,section,stop,kind
Topic 3,hear,listen,masterpiece,legend,year,vibe,people,fan,recuerdo,shit,omg,beat,wait,ser,moment
Topic 4,song,love,time,remember,listen,dance,kid,lyric,life,day,fly,mean,forget,sing,mind
Topic 5,view,come,people,video,youtube,check,fact,search,reach,party,recommend,reason,hell,talk,count
Topic 6,like,nostalgia,play,school,day,dance,time,break,live,style,end,gold,person,everytime,reddit
Topic 7,know,pop,say,guy,man,understand,world,need,think,word,friend,realize,history,wonder,idea
Topic 8,let,feel,video,que,era,todo,song,like,king,edit,internet,die,generation,population,por
Topic 9,music,song,childhood,world,way,lol,language,tell,agree,fun,tinha,understand,industry,cause,singer


In [21]:
Topics = ["Style","Childhood","Music","Fun","Diverse culture", 
          "Nostalgia", "Exciting", "실력", "남녀노소", "SNS"]
df_topic_keywords["Topics"]=Topics
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14,Topics
Topic 0,watch,style,make,memory,think,dance,video,bring,thank,este,imagine,psy,remember,change,banger,Style
Topic 1,year,song,kpop,listen,know,use,dance,believe,remember,release,miss,time,today,day,lot,Childhood
Topic 2,psy,comment,hit,style,girl,look,thing,lady,want,styleeh,end,baby,section,stop,kind,Music
Topic 3,hear,listen,masterpiece,legend,year,vibe,people,fan,recuerdo,shit,omg,beat,wait,ser,moment,Fun
Topic 4,song,love,time,remember,listen,dance,kid,lyric,life,day,fly,mean,forget,sing,mind,Diverse culture
Topic 5,view,come,people,video,youtube,check,fact,search,reach,party,recommend,reason,hell,talk,count,Nostalgia
Topic 6,like,nostalgia,play,school,day,dance,time,break,live,style,end,gold,person,everytime,reddit,Exciting
Topic 7,know,pop,say,guy,man,understand,world,need,think,word,friend,realize,history,wonder,idea,실력
Topic 8,let,feel,video,que,era,todo,song,like,king,edit,internet,die,generation,population,por,남녀노소
Topic 9,music,song,childhood,world,way,lol,language,tell,agree,fun,tinha,understand,industry,cause,singer,SNS
